<a href="https://colab.research.google.com/github/Azharsayyed5/Chat-Bot-Tensorflow/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk

In [2]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [3]:
pip install tflearn

In [4]:
pip uninstall tensorflow -y

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0


In [5]:
pip install tensorflow==1.15

  Using cached https://files.pythonhosted.org/packages/92/2b/e3af15221da9ff323521565fa3324b0d7c7c5b1d7a8ca66984c8d59cb0ce/tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl


In [6]:
import numpy
import tflearn
import tensorflow
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
words = []
labels = []
docs_x = []
docs_y = []

In [9]:
with open("train.json") as file:
    data = json.load(file)

In [10]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [11]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [12]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)
  

training = numpy.array(training)
output = numpy.array(output)

In [13]:
with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [14]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])

net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 18999  | total loss: 0.01206 | time: 0.144s
| Adam | epoch: 1000 | loss: 0.01206 - acc: 0.9855 -- iter: 144/149
Training Step: 19000  | total loss: 0.01089 | time: 0.155s
| Adam | epoch: 1000 | loss: 0.01089 - acc: 0.9869 -- iter: 149/149
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


# TALK TO THE **BOT**

In [15]:
with open("train.json") as file:
    data = json.load(file)
with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

In [16]:
tensorflow.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
model = tflearn.DNN(net)
model.load("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [17]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [20]:
import random

In [21]:
def chat(inp):
    results = model.predict([bag_of_words(inp, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
        if tg['tag'] == tag:
            response = tg['responses']

            print(random.choice(response))

In [ ]:
print("Start talking with the bot (type quit to stop)!")
while True:
    inp = input("You: ")
    if inp.lower() == "quit":
        print("bye")
        break
    chat(inp)

Start talking with the bot (type quit to stop)!
